<a href="https://colab.research.google.com/github/mashyko/NLP_BERT_Transformers/blob/master/japanese_vectorize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# word2vec、fastTextを用いた日本語単語のベクトル表現の実装

日本語の単語をword2vecもしくはfastTextを使用してベクトル化して、単語間の類似度を検証

「つくりながら学ぶ! PyTorchによる発展ディープラーニング」（小川雄太郎、マイナビ出版 ）」で提供されているコードを使用させていただきました。


使用するMecabおよびデータを用意します


In [1]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [2]:
!mecab-config --dicdir


/usr/lib/x86_64-linux-gnu/mecab/dic


In [3]:
!cat /etc/mecabrc

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /var/lib/mecab/dic/debian

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



文書を読み込んで、分かち書き、データセット作成

前処理と分かち書きをし、最後にデータセットを作成する部分を実装


In [0]:
# 単語分割にはMecab＋NEologdを使用
import MeCab

m_t = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret



# 前処理として正規化をする関数を定義
import re

def preprocessing_text(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    return text


# 前処理とMecabの単語分割を合わせた関数を定義する


def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_mecab(text)  # Mecabの単語分割

    return ret


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torchtext

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)


# フォルダ「data」から各tsvファイルを読み込みます
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='/content/drive/My Drive/data/tsv/', train='train.tsv',
    validation='val.tsv', test='test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

 単語のベクトル化：word2vec 
 

以下の日本語のfasttextの学習済みベクトルをダウンロードします。

https://www.nlp.ecei.tohoku.ac.jp/~m-suzuki/bert-japanese/BERT-base_mecab-ipadic-char-4k_whole-word-mask.tar.xz

My Drive/data/に解凍・展開してください。

url = "http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2"

save_path = "/content/drive/My Drive/data/20170201.tar.bz2"

if not os.path.exists(save_path):

    urllib.request.urlretrieve(url, save_path)

 tarファイルを読み込み

tar = tarfile.open('/content/drive/My Drive/data/20170201.tar.bz2', 'r|bz2')

tar.extractall('/content/drive/My Drive/data/')  # 解凍

tar.close()  # ファイルをクローズ

My Driveのフォルダ「data」内にフォルダ「entity_vector」というものができ、その中に「entity_vector.model.bin」というファイルができています。

In [8]:
# 事前インストール
pip install gensim

In [9]:
# そのままではtorchtextで読み込めないので、gensimライブラリを使用して、

from gensim.models import KeyedVectors

# 一度gensimライブラリで読み込んで、word2vecのformatで保存する
model = KeyedVectors.load_word2vec_format(
    '/content/drive/My Drive/data/entity_vector.model.bin', binary=True)

# Word2Vecのformatで保存し直します

#保存（時間がかかります、10分弱）
model.wv.save_word2vec_format('./japanese_word2vec_vectors.vec')


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()


In [10]:
# torchtextで単語ベクトルとして読み込みます
from torchtext.vocab import Vectors

japanese_word2vec_vectors = Vectors(
    name='./japanese_word2vec_vectors.vec')

# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_word2vec_vectors.dim)
print("単語数：", len(japanese_word2vec_vectors.itos))


100%|█████████▉| 1014952/1015474 [01:38<00:00, 9821.31it/s]

1単語を表現する次元数： 200
単語数： 1015474


In [11]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 49個の単語が200次元のベクトルで表現されている
TEXT.vocab.vectors


torch.Size([73, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.5917,  0.4508, -0.5694,  ...,  0.4193,  0.7276,  0.0932],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4007, -3.0376,  1.4939,  ..., -1.1628,  1.3272,  1.5678],
        [-0.6148, -0.3318, -1.3326,  ..., -0.0495, -2.4379, -0.0824]])

In [12]:
# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


defaultdict(<function torchtext.vocab._default_unk_index>,
            {'0': 29,
             '<pad>': 1,
             '<unk>': 0,
             '、': 2,
             '。': 4,
             '「': 30,
             '」': 31,
             'ある': 16,
             'い': 13,
             'いる': 17,
             'か': 18,
             'が': 5,
             'き': 32,
             'この': 19,
             'さ': 20,
             'し': 14,
             'しまう': 33,
             'する': 21,
             'その': 34,
             'た': 15,
             'て': 7,
             'で': 22,
             'でし': 35,
             'と': 9,
             'とても': 36,
             'な': 10,
             'に': 11,
             'に対して': 37,
             'の': 3,
             'は': 6,
             'まし': 23,
             'ます': 24,
             'れ': 25,
             'を': 8,
             'アニメ映画': 38,
             'クラス': 39,
             'ネガティブ': 40,
             'ポジティブ': 41,
             'モデル': 42,
             'レビュー': 43,
             '人工知能': 44,
    

In [13]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[55] - \
    TEXT.vocab.vectors[53] + TEXT.vocab.vectors[65]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[54], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[63], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[64], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[61], dim=0))


女王 tensor(0.3840)
王 tensor(0.3669)
王子 tensor(0.5489)
機械学習 tensor(-0.1404)


姫 - 女性 + 男性　を計算すると狙った通り、王子がもっとも近い結果になりました

word2vecより進歩したベクトル化手法であるfastTextによる単語のベクトル表現を使用します。

日本語の学習モデルを以下のサイトにて公開されているので、使用させていただきます。


vector_neologd.zipのダウンロード
https://drive.google.com/open?id=0ByFQ96A4DgSPUm9wVWRLdm5qbmc

ダウンロードしたら、My Drive/vector_neologd に配置してください。

In [14]:
# torchtextで単語ベクトルとして読み込みます
# word2vecとは異なり、すぐに読み込めます

from torchtext.vocab import Vectors

japanese_fasttext_vectors = Vectors(name='/content/drive/My Drive/data/vector_neologd/model.vec')

                                    
# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_fasttext_vectors.dim)
print("単語数：", len(japanese_fasttext_vectors.itos))



  0%|          | 0/351122 [00:00<?, ?it/s]Skipping token b'351122' with 1-dimensional vector [b'300']; likely a header

100%|█████████▉| 350427/351122 [00:47<00:00, 7285.47it/s]

1単語を表現する次元数： 300
単語数： 351122


In [15]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_fasttext_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 52個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


torch.Size([73, 300])


defaultdict(<function torchtext.vocab._default_unk_index>,
            {'0': 29,
             '<pad>': 1,
             '<unk>': 0,
             '、': 2,
             '。': 4,
             '「': 30,
             '」': 31,
             'ある': 16,
             'い': 13,
             'いる': 17,
             'か': 18,
             'が': 5,
             'き': 32,
             'この': 19,
             'さ': 20,
             'し': 14,
             'しまう': 33,
             'する': 21,
             'その': 34,
             'た': 15,
             'て': 7,
             'で': 22,
             'でし': 35,
             'と': 9,
             'とても': 36,
             'な': 10,
             'に': 11,
             'に対して': 37,
             'の': 3,
             'は': 6,
             'まし': 23,
             'ます': 24,
             'れ': 25,
             'を': 8,
             'アニメ映画': 38,
             'クラス': 39,
             'ネガティブ': 40,
             'ポジティブ': 41,
             'モデル': 42,
             'レビュー': 43,
             '人工知能': 44,
    

In [16]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[55] - \
    TEXT.vocab.vectors[53] + TEXT.vocab.vectors[65]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[54], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[63], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[64], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[61], dim=0))


女王 tensor(0.3650)
王 tensor(0.3461)
王子 tensor(0.5531)
機械学習 tensor(0.0952)


姫 - 女性 + 男性　を計算すると狙った通り、王子がもっとも近い結果になりました

以上